## Feature Engineering

The next couple steps we want to take are 1. adding a score value to rank players which we'll do using the add_player_value function and 2. assigning weights to the past three years worth of player data. Here's also the chance where we can perhaps think of adding more variables? E.g. history of injuries, starter or not, advanced stats?

In [1]:
import pandas as pd
import os
import utils
import warnings

from scipy.stats import zscore 
from tqdm.notebook import tqdm
from constants import DATA_DIR
from constants import PUNT_TYPES
warnings.filterwarnings('ignore')

Player value is a sum of the season's zcores for each of the 9-cat stats. FG% and FT% are averaged with the zscores of attempted FG and FT to reward players who shoot more. Turnover z-scores are inverted. Although it's not perfect, it provides a holistic score of a player. The punting aspect comes from removing a specified stat from the player value calculation. Ideally there'd be an actual fantasy score calculation such as with daily fantasy, but we'll have to make do with this for now.

In [2]:
def add_player_value(df, punt):
    all_stats = ['FG%','FGA','FT%','FTA','3P','PTS','TRB','AST','STL','BLK','TOV']
    stats = [x for x in all_stats if x not in punt]
    z_scores = pd.DataFrame(zscore(df[stats], axis=0, nan_policy='omit'), columns=stats)
    #TOV, FG%, & FT% need to be adjusted to include volume
    if 'TOV' not in punt:
        z_scores['TOV'] = z_scores['TOV'].apply(lambda x: x * -1)
    if 'FG%' not in punt:
        z_scores['FG%'] = z_scores[['FG%','FGA']].mean(axis=1)
    if 'FT%' not in punt:
        z_scores['FT%'] = z_scores[['FT%','FTA']].mean(axis=1)
    
    #drop FGA and FTA
    z_scores.drop(columns=['FGA','FTA'], inplace=True)

    #adjust stats to increase if they were punted -> otherwise base will always be higher
    #not really sure what value we'll use here, but we can try multiplying by 9/8
    if len(punt) != 0:
        z_scores = z_scores * 9 / 8

    z_scores['VALUE'] = z_scores.sum(axis=1)
    result = pd.concat([df, z_scores['VALUE']], axis=1)

    return result

After adding player value, we'll also want to add weightings to the past three seasons worth of player data. We'll do this four different ways, with three methods giving more weight to more recent seasons. 

1. Base - all seasons are weighted evenly. 

2. Linear - weights grow by one e.g. 1/6 assigned to the first, 2/6 assigned to the second, 3/6 assigned to the third.

3. Square root - weights grow by one but are square rooted

4. Quadratic - weights grow by one but are squared e.g. 1/14 assigned to the first, 4/14 assigned the second, 9/14 assigned to the third.

In [3]:
# combine data
seasons = ['2009-10','2010-11','2011-12','2012-13','2013-14','2014-15', 
           '2015-16','2016-17','2017-18','2018-19','2019-20','2020-21']
merged = pd.DataFrame()
for punt in PUNT_TYPES:
    # ADD PLAYER VALUE FOR EACH PUNTING STAT
    if punt == []:
        punt_name = 'Base'
    else:
        punt_name = '+'.join(punt)
    for season in seasons:
        df = pd.read_csv('~/Desktop/NBA-Fantasy-Draft/data/PlayerStats/'+season+'.csv')
        df = add_player_value(df, punt)
        directory = os.path.join(DATA_DIR,punt_name,'Value')
        if not os.path.exists(directory):
            os.makedirs(directory)
        df.to_csv(os.path.join(directory,season+'.csv'), index=False)
    
    # CREATE SQRT, LINEAR, QUAD, WEIGHTS FOR EACH DATA
    merged = utils.csv_concatenate(os.path.join(DATA_DIR,punt_name,'Value'))
    weightings = ['base','linear','quad','sqrt']
    weight_seasons = seasons[3:]
    for weighting in tqdm(weightings):
        for weight_season in weight_seasons:
            df = utils.weigh_data(weighting, weight_season, merged, False)
            directory = os.path.join(DATA_DIR,punt_name,'Weighted',weighting)
            if not os.path.exists(directory):
                os.makedirs(directory)
            df.to_csv(os.path.join(DATA_DIR,punt_name,'Weighted',weighting,weight_season+'.csv'), index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/466 [00:00<?, ?it/s]

  0%|          | 0/478 [00:00<?, ?it/s]

  0%|          | 0/491 [00:00<?, ?it/s]

  0%|          | 0/475 [00:00<?, ?it/s]

...

  0%|          | 0/482 [00:00<?, ?it/s]

  0%|          | 0/535 [00:00<?, ?it/s]

  0%|          | 0/526 [00:00<?, ?it/s]

  0%|          | 0/526 [00:00<?, ?it/s]

  0%|          | 0/536 [00:00<?, ?it/s]